# Postprocessing of the data of bulk Aluminium

This notebook will serve as a template to post-process the different inputs of the FT of $\chi^0(\boldsymbol{q}, \boldsymbol{G}, \boldsymbol{G'})$. Different test will be experimented to test convergence and accuracy of the results. This notebook deals in parallel with primitive and conventional cells.

Visualization tools will also be available to go further in the analysis.



In [49]:
import numpy as np
import math 
import cmath
import abipy
from abipy.electrons.scr import ScrFile
import time
import matplotlib.pyplot as plt
import pointcloud as pc
from scipy.interpolate import RegularGridInterpolator
import plotly.graph_objects as go
%load_ext autoreload
%autoreload 2
import Fourier_tool as Ft
import XGChi0 
import MatrixCharacteristic
from abipy import abilab
import abipy.data as abidata
from abipy.abilab import Structure


chi0_symb = '\\chi^0(\\boldsymbol{q}, \\boldsymbol{G}_1, \\boldsymbol{G}_2)'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## List of Abinit files (pay attention to always specify ngkpt and nsym with the filename): 

- Al_Bulk_eps17_SUS.nc (generic file : nband = 150, ecuteps = 17, ngkpt = [4, 4, 4], nsym = 0)

- Al_Bulk_nkpt4_nbando_DS1_SUS.nc, Al_Bulk_nkpt4_nbando_DS2_SUS.nc, Al_Bulk_nkpt4_nbando_DS3_SUS.nc, Al_Bulk_nkpt4_nbando_DS4_SUS.nc, Al_Bulk_nkpt4_nbando_DS5_SUS.nc (by increasing number of bands [150 -> 300] with ecuteps = 9, ngkpt = [4, 4, 4], nsym = 0)

- Al_Bulk_nkpt4o_DS1_SUS.nc, Al_Bulk_nkpt4o_DS2_SUS.nc, Al_Bulk_nkpt4o_DS3_SUS.nc, Al_Bulk_nkpt4o_DS4_SUS.nc, Al_Bulk_nkpt4o_DS5_SUS.nc, Al_Bulk_nkpt4o_DS6_SUS.nc (by increasing ecuteps [5 - 25] with nband = 150, ngkpt = [4, 4, 4], nsym = 0)

- Al_Bulko_DS1_SUS.nc (Conv cell, ngkpt = [4, 4, 4], nsym = 0)

- Al_Bulk_nkpt6o_DS1_SUS.nc (ngkpt = [6, 6, 6], ecuteps < 6, nsym = 0)

- Al_Bulk_nkpt8o_DS1_SUS.nc, Al_Bulk_nkpt8o_DS2_SUS.nc (ngkpt = [8, 8, 8], ecuteps = 3.6, 17, nsym = 0)

- Al_Bulk_nkpt10o_DS1_SUS.nc (ngkpt = [10, 10, 10], ecuteps = 17, nsym = 0)

- Al_Bulk_nsym0o_DS1_SUS.nc, Al_Bulk_nsym1o_DS1_SUS.nc (ngkpt = [4, 4, 4], ecuteps = 3.6, nsym =[0,1])

## Choose the file you want to analyse

Don't forget to specify the ngkpt and nsym variables from the data in the list above

In [21]:
filename = "Al_Bulk_nkpt4o_DS5_SUS.nc"
ngkpt = [4, 4, 4] #has to be entered manually from the data given above
nsym = 0

In [ ]:
## Get all the information on the structure and visualize the supercell

In [4]:
structure=abipy.core.structure.Structure.from_file(filename)
structure

Structure Summary
Lattice
    abc : 2.8535866547158504 2.8535866547158504 2.8535866547158504
 angles : 59.999999999999986 59.99999999999999 59.99999999999999
 volume : 16.430780655166938
      A : 1.6475190232561199 0.0 2.3299437469564794
      B : -0.8237595116280599 1.426793327357925 2.3299437469564794
      C : -0.8237595116280599 -1.426793327357925 2.3299437469564794
PeriodicSite: Al (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]

In [7]:
structure.visualize("vesta")

Writing data to: .xsf with fmt: xsf
Executing MacOSx open command: open -a vesta --args   /Users/acloots/Documents/GitHub/Thesis_SURFASCOPE/.xsf


0

In [12]:
dfs = abilab.dataframes_from_structures([structure], index=[".nc"])
dfs.lattice

,formula,natom,alpha,beta,gamma,a,b,c,volume,abispg_num,spglib_symb,spglib_num,spglib_lattice_type
.nc,Al1,1,60.0,60.0,60.0,2.853587,2.853587,2.853587,16.430781,0,Fm-3m,225,cubic


In [20]:
dict_struct = structure.to_abivars()
natom = dict_struct["natom"]
xred = dict_struct["xred"]

In [19]:
lattice = structure.lattice.matrix
A, B, C = lattice[0], lattice[1], lattice[2]

In [33]:
XGChi0.Supercell_Vis(A, B, C, ngkpt, natom, xred)

## Lists of the generic information of the $\chi^0(\boldsymbol{q}, \boldsymbol{G}, \boldsymbol{G'})$ given by abinit

In [52]:
sus_ncfile = ScrFile(filename)
nw=sus_ncfile.reader.nw
nwr = sus_ncfile.reader.nrew
nwi = sus_ncfile.reader.nimw
ng=sus_ncfile.reader.ng 
kpoints=sus_ncfile.reader.kpoints
nkpt = len(kpoints)
chi0=sus_ncfile.reader.read_wggmat(kpoints[0])
chi0_mat = chi0.wggmat
print("Data for filename\nNumber of frequencies = ",nw, "(",nwr, "real frequencies and ", nwi, "imaginary frequencies)\nNumber of K-points = ", nkpt,"\nNumber of G-vector in the G-sphere = ", ng,"\n$%s$"%chi0_symb,'=',chi0_mat.shape)

Data for filename
Number of frequencies =  14 ( 10 real frequencies and  4 imaginary frequencies)
Number of K-points =  8 
Number of G-vector in the G-sphere =  531 
$\chi^0oldsymbol{q},oldsymbol{G}_1,oldsymbol{G}_2)$ = (14, 531, 531)


In [ ]:
chi0GG, ind_qG_to_vec, n1, n2, n3, ind_q_to_vec, ind_G_to_vec, G, nk = XGChi0.Build_Chi0GG("Al_Bulk_nkpt4o_DS5_SUS.nc", "FromSym", omega = 0)